# Annex A Execution

A notebook implementing the execution rules from KerML Annex A with PyMBE.

In [ ]:
import json
import pymbe.api as pm

import copy

from importlib import resources as lib_resources

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import *

from pymbe.query.metamodel_navigator import is_type_undefined_mult, \
                                    is_multiplicity_one, \
                                    is_multiplicity_specific_finite, \
                                    get_finite_multiplicity_types, \
                                    identify_connectors_one_side, \
                                    get_lower_multiplicity, \
                                    get_upper_multiplicity, \
                                    does_behavior_have_write_features, \
                                    get_most_specific_feature_type, \
                                    has_type_named, \
                                    get_effective_lower_multiplicity, \
                                    get_feature_bound_values, \
                                    get_more_general_types

from pymbe.metamodel import derive_inherited_featurememberships

from pymbe.text_concrete_syntax import serialize_kerml_atom

from pymbe.interpretation.working_maps import FeatureTypeWorkingMap
from pymbe.interpretation.execute_kerml_atoms import KermlForwardExecutor

from uuid import uuid4

## Key Helpers for the Algorithm

These helpers are yet to be implemented in the core of the Python tool and thus need to be more spelled out.

### Check for Connectors to Features

In [ ]:
def is_feature_connected(feature):
    print(f"...Inspecting {feature.declaredName} for connector references.")
    if hasattr(feature, "reverseReferenceSubsetting"):
        print(f"...Found link to connector end {feature.reverseReferenceSubsetting[0]}")
        return True
    else:
        print(f"...Found no reverse edge outgoing to connector end.")
        return False

# Load up Kernel Libraries

Load up the model libraries into memory so that key features for subsetting can be found.

In [ ]:
library = "KernelLibrary"

library_model = None

with lib_resources.path("pymbe.static_data", "KernelLibrary.json") as lib_data:
    library_model = pm.Model.load_from_post_file(lib_data)

## Routines for Execution

The following sections are focused on solving the problem of mapping values to KerML types in the model. The approach taken here is to find one legal set of values for types in the model via an approach where the program will walk straight ahead in the model, deriving values as it goes. This approach is called "execution" here.

In [ ]:
def print_values_dictionary(model, values_dict):
    print_string = ""
    for k, v in values_dict.items():
        print_string = print_string + f">>>Key {model.get_element(k)} ({k}) has values {v}\n"
        
    print(print_string)

In [ ]:
def pretty_print_steps_log(builder_log):
    # helper to print the log in a way you can read it in the Jupyter notebook easily
    print("\n".join(builder_log))

## Atom Metadata Load

Bring up the Atom metadata.

filename = "A-2-Atoms"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

atoms_data = pm.Model.load_from_post_file(json_file)
atoms_data

## Annex A.3.2 Without Connectors Case

In [ ]:
filename = "A-3-2-WithoutConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

without_connectors_data = pm.Model.load_from_post_file(json_file)
without_connectors_data

without_connectors_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in without_connectors_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
without_connectors_to_execute_classifiers = \
    [ele for ele in packages[0].throughOwningMembership if ele._metatype == 'Classifier']

In [ ]:
without_connectors_executor = KermlForwardExecutor(without_connectors_data, packages[1])

In [ ]:
packages[0].throughOwningMembership

In [ ]:
packages[0].throughOwningMembership[1].throughSubclassification

In [ ]:
packages[0].throughOwningMembership[2].basic_name

In [ ]:
without_connectors_executor.execute_from_classifier(packages[0].throughOwningMembership[1])

In [ ]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

In [ ]:
print(without_connectors_executor._traversal_log)

In [ ]:
pretty_print_steps_log(without_connectors_executor._builder_log['Bicycle «Classifier».[]'])

In [ ]:
print("\n".join(without_connectors_executor._value_map_log))

## Annex A.3.3 One-To-One Connectors Case

In [ ]:
filename = "A-3-3-OneToOneConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

one_2_one_connectors_data = pm.Model.load_from_post_file(json_file)
one_2_one_connectors_data

one_2_one_connectors_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in one_2_one_connectors_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
packages[2].throughOwningMembership

In [ ]:
packages[2].throughOwningMembership[1].throughFeatureMembership[0].throughEndFeatureMembership

In [ ]:
one_2_one_connectors_executor = KermlForwardExecutor(one_2_one_connectors_data, packages[3])

In [ ]:
one_2_one_connectors_executor.execute_from_classifier(packages[2].throughOwningMembership[1])

In [ ]:
for item in packages[3].throughOwningMembership:
    print(serialize_kerml_atom(item))

In [ ]:
packages[3].throughOwningMembership

In [ ]:
packages[3].throughOwningMembership[4].throughSubclassification

In [ ]:
print(without_connectors_executor._traversal_log)

In [ ]:
list(without_connectors_executor._builder_log.keys())

In [ ]:
pretty_print_steps_log(without_connectors_executor._builder_log['Bicycle «Classifier».[]'])

In [ ]:
print("\n".join(without_connectors_executor._value_map_log))

## Annex A.3.6 Timing for Behaviors, Sequences

In [ ]:
filename = "A-3-6-Sequences"
filename = "A-3-6-aSequences-User-Only"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

sequences_data = pm.Model.load_from_post_file(json_file)
sequences_data

sequences_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in sequences_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
packages[0].throughOwningMembership

In [ ]:
packages[0].throughOwningMembership[1].throughFeatureMembership

In [ ]:
packages[0].throughOwningMembership[1].throughFeatureMembership[0].throughSubsetting[0]._derived

In [ ]:
sequences_executor = KermlForwardExecutor(sequences_data, packages[1])

In [ ]:
sequences_executor.execute_from_classifier(packages[0].throughOwningMembership[1])

In [ ]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

In [ ]:
packages[1].throughOwningMembership

In [ ]:
print(sequences_executor._traversal_log)

In [ ]:
list(sequences_executor._builder_log.keys())

In [ ]:
pretty_print_steps_log(sequences_executor._builder_log['dry: Dry «Step».[]'])

In [ ]:
print("\n".join(sequences_executor._value_map_log))

## Annex A.3.8 Feature Value Changes

In [ ]:
filename = "A-3-8-ChangingFeatureValues"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

values_data = pm.Model.load_from_post_file(json_file)
values_data

values_data.reference_other_model(library_model)

In [ ]:
packages = [ele for ele in values_data.elements.values() if ele._metatype == 'Package']
packages

In [ ]:
packages[1]

In [ ]:
values_executor = KermlForwardExecutor(values_data, packages[1])

In [ ]:
packages[0]

In [ ]:
has_type_named(packages[0].throughOwningMembership[3].throughFeatureMembership[1], "FeatureWritePerformance")

In [ ]:
packages[0].throughOwningMembership[3].throughFeatureMembership[1].throughFeatureMembership

In [ ]:
packages[0].throughOwningMembership[3].throughFeatureMembership[1].throughFeatureMembership[0].throughFeatureMembership[0].throughFeatureMembership[0].throughSubsetting

In [ ]:
values_executor.execute_from_classifier(packages[0].throughOwningMembership[1])

In [ ]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

In [ ]:
print(values_executor._traversal_log)

In [ ]:
list(values_executor._builder_log.keys())

In [ ]:
pretty_print_steps_log(values_executor._builder_log['paint: Paint «Step».[]'])

In [ ]:
print("\n".join(values_executor._value_map_log))